In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import qutip as qt
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
import qiskit
from itertools import product
from typing import List, Dict, Tuple

In [6]:
qt.__version__

'5.1.1'

In [7]:
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
I = qt.identity(2)
Had = qt.core.gates.snot()
q0 = qt.basis(2,0)
q1 = qt.basis(2,1)
pl = Had*q0
pi = np.pi
sqrt=np.sqrt

### Functions for QAOA Grover's:
- phase_operator(C,gamma) : is a unitary generated by an operator C, in standard Grover's approach it is a projector to a bitstring.
- mixer(beta, NQ) is a standard QAOA mixer, for the Grover's beta angles are set to be $\pi/N$, where $N=$number of qubits
- W_op(C, gamma, NQ) is the total single block of the QAOA grover's

In [3]:
def phase_operator(C, gamma):
    return (-1j*C*gamma).expm()

def mixer(beta, NQ):
    M = [(-1j*beta*X).expm()]*NQ
    return qt.tensor(M)

def W_op(C, gamma, NQ):
    beta = np.pi/NQ
    W = mixer(beta, NQ ) * phase_operator(C, -gamma)*mixer(beta, NQ ) * phase_operator(C, gamma)
    return W

In [8]:
def hamiltonian_ising(Jij_dict, h, NQ = 3):
    H = qt.tensor([I]*NQ)*0
    for edge, Jij in Jij_dict.items():
        Z_temp = [I]*NQ
        Z_temp[edge[0]] = Z
        Z_temp[edge[1]] = Z
        H += Jij * qt.tensor(Z_temp)
    for idx in range(NQ):
        Z_local = [I] * NQ
        Z_local[idx] = Z
        H += h[idx] * qt.tensor(Z_local)
    return H